# TP 3 "Régression polynomiale / Régularisation"

*M2 IWOCS, Apprentissage automatique*

## Données du problème
Télécharger l’archive correspondant aux données de l’exercice 3/TP 3, décompactez-la. cette archive contient deux fichiers, un pour les données d'entrée x qui sont ici à une dimension, et l’autre pour les données de sorties y à une dimension également. Tracer un graphique représentant le nuage de points défini par ces données dans le style du celui qui est représenté dans la figure suivante.
<img src="tp3-enonce-fig1.png" width="400"/>

In [ ]:
import numpy as np 

x = np.loadtxt('ex3dat/ex3Linx')
y = np.loadtxt('ex3dat/ex3Liny')

print("x: \n",x)
print("y: \n",y)

: 

## Analyse du problème
On propose de modéliser ces données par une fonction hypothèse polynomiale de degré 5 :

$$
h_{\theta}(x) = \theta_0 + \theta_1 x + \theta_2 x^2 + + \theta_3 x^3 + \theta_4 x^4 + \theta_5 x^5
$$

Comme expliqué dans le cours, cela revient à considérer 6 descripteurs constituant les différentes puissances de x dans le polynôme. Le problème reste cependant linéaire par rapport à ces descripteurs.

Le degré du polynôme étant 5 et le nombre de points initiaux 7, le risque de sur-apprentissage (over-fitting) est grand. Il est donc nécessaire de *régulariser* le modèle.

Pour cela on va corriger le calcul de la fonction de coût en utilisant une variante de la régularisation de Ridge vu en cours et qui s'écrit :

$$
J(\theta) = \frac{1}{2m}\sum_{i=1}^{m}\left(h_\theta(x_i)-y_i\right)^2+\lambda \sum_{j=1}^{n}\theta_j^2
$$

Le paramètre de régularisation $\lambda$ va permettre d’éviter des valeurs importantes de $\theta$ qui feraient augmenter la valeur de la fonction de coût. A noter que la composantes $\theta_O$ n’intervient pas dans la sommation devant $\lambda$.


## Modification des formulations matricielles pour intégrer le terme de régularisation de type Ridge
L'introduction de la régularisation intervient dans la fonction de calcul du coût et dans le calcul de la minimisation de la fonction coût avec la méthode de gradient

### Calcul du coût

Formule du coût avec la régularisation proposée dans l'exercice (variante de la régularisation de Ridge) :
$$
J(\theta) = \frac{1}{2m}\sum_{i=1}^{m}\left(h_\theta(x_i)-y_i\right)^2+\lambda \sum_{j=1}^{n}\theta_j^2
$$

Comme vu précédemment, la première partie de cette formule (avant le terme de régularisation)
$$
\frac{1}{2m}\sum_{i=1}^{m}\left(h_\theta(x_i)-y_i\right)^2
$$
s'écrit en notation matricielle
$$
\frac{1}{2m}\left[h_\theta(X)-Y\right]^t \left[h_\theta(X)-Y\right]
$$
avec
\begin{align}
 \text{avec} \quad
 h_\theta(X) &= X \theta\\
             &= 
             \begin{bmatrix}
             1 & x_{11} & \cdots & x_{1n}\\
             \vdots & \vdots & & \vdots\\
             1 & x_{m1} & \cdots & x_{mn}\\
             \end{bmatrix}
             \begin{bmatrix}
             \theta_0\\
             \vdots\\
             \theta_n
             \end{bmatrix}\\
\text{et} \quad 
Y &= \begin{bmatrix}
      y_1\\
      \vdots\\
      y_m
      \end{bmatrix}
\end{align}

On doit alors ajouter à la fonction coût, le terme $\lambda \sum_{j=1}^{n}\theta_j^2$ (attention : on a exclu $\theta_0$). En notation matricielle, cette expression s'écrit

$$
\lambda [\theta_-]^t[\theta_-] \quad \text{où} \quad 
\theta_- = \begin{bmatrix}
                0\\
                \theta_1\\
                \vdots\\
                \theta_n
            \end{bmatrix}
$$
Donc au final $J(\theta)$ s'écrit en notation matricielle
$$
J(\theta) = \frac{1}{2m}\left[X \theta -Y \right]^t \left[X \theta - Y\right] + \lambda [\theta_-]^t[\theta_-]
$$

###  Calcul de la minimisation de la fonction de coût régularisée avec la méthode de gradiant

A chaque itération, on calcule une nouvelle valeur de $\theta^*$ à partir de $\theta$ telle que $J(\theta^*) < J(\theta)$.

La méthode de gradient correspond au calcul suivant :
$$
\theta^* = \theta - \alpha \nabla J(\theta)
$$

\begin{cases}
\theta_0^* &= \theta_0 - \alpha \displaystyle{\frac{\partial}{\partial \theta_0}} J(\theta)\\
\vdots\\
\theta_k^* &= \theta_k - \alpha \displaystyle{\frac{\partial}{\partial \theta_k}} J(\theta)\\
\vdots\\
\theta_k^* &= \theta_k - \alpha \displaystyle{\frac{\partial}{\partial \theta_k}} J(\theta)
\end{cases}

\begin{align}
\displaystyle{\frac{\partial}{\partial \theta_k}} J(\theta) &= 
    \displaystyle{\frac{\partial}{\partial \theta_k}} 
        \left( \frac{1}{2m}\sum_{i=1}^{m}\left(h_\theta(x_i)-y_i\right)^2+\lambda \sum_{j=1}^{n}\theta_j^2 \right)\\
     &= \frac{1}{m}\left(\sum_{i=1}^{m}\left(h_\theta(x_i)-y_i\right)x_{ik}\right) + 2 \lambda \theta_k
\end{align}
Le dernier terme du membre droit de l'équation $2 \lambda \theta_k$ vaut 0 si $k=0$ en raison de la formule de régularisation utilisée.

La formulation matricielle s'écrit ainsi (en utilisant que $h_\theta(X) = X \theta$) :
$$
\theta^* = \theta - \alpha \left[ \frac{1}{m} X^t [X \theta - Y] + 2 \lambda \theta_- \right]
$$

##  Discussion sur le choix de la régularisation de type Ridge utilisée
Afin de comprendre l'intérêt du choix de la régularisation utilisée ici, à savoir une régularisation Ridge sans prendre le premier terme $\theta_0$, on pourra comparer les résultats que l'on obtiendra dans la mise en ouvre proposée ci-dessous avec une régularisation de Ridge complète et intégrant $\theta_0$. 

## Mise en oeuvre

Il s’agit d’adapter l’algorithme de régression linéaire écrite sous forme matricielle et modifié à partir des formules précédents pour intégrer le terme de régularisation. 

Comme cela est expliqué dans le cours, on va ramener le problème de regression polynomiale univarié à un problème de régression linéaire multivarié, en constituant une matrice X de la façon suivante à partir du vecteur colonne x de la donnée initiale :


`X=np.block([[xone,x,x**2,x**3,x**4,x**5]])`

Réaliser cette régression pour $\lambda = 0, \lambda = 1, \lambda = 10$, puis tracer la courbe résultante sur le même graphique que les données. On doit alors obtenir une figure comparable à la suivante :

<img src="tp3-enonce-fig2.png" width="900"/>


In [ ]:
def gradient(X,Y, alpha = 0.07, nb_iterations_max = 2500, e = 1e-6): 
    # Initialisation 
    theta = np.zeros(X.shape[1])
        
    for i in range(nb_iterations_max): 
        theta_old = theta.copy()    # on copie la valeur de theta 
        
        theta = iteration(theta,X,Y,alpha) # calcule de la nouvelle valeur optimal du vecteur theta
        
        # Afficher le progrès
        if i % 100 == 0:  
            print(f"Itération {i:4d}: Theta = {theta}")
            
        # Critère d'arrets (différence relative)
        if i > 0: # condition pour eviter les division par zéro à la première itération 
           
           # Calcule |θ^(n+1) - θ^(n)| 
           diff_theta = np.linalg.norm(theta-theta_old)
           
           #Calcule de theta |θ^(n)|
           norm_theta_old = np.linalg.norm(theta_old)
            
            # Éviter la division par zéro  
           if norm_theta_old > 1e-10: 
            # Calcule du critère d'arrêt 
              diff_rel = diff_theta / norm_theta_old
              
              # Vérifier le critère d'arrêt
              if diff_rel < e :
                  print(f"Critère d'arrêt déclencher ! Fin de la boucle") 
                  break
            
    else : 
        print("Nombre d'itération maximum atteint ! ")
                   
    print(f"Theta optimal :{theta}")
    
    return theta      
            
# Utilisation 
theta_optimal = gradient(X,y,alpha = 0.07, e=1e-6)            

## Compromis biais-variance

A partir de l’article wikipedia sur le compromis biais-variance en apprentissage : [https://fr.wikipedia.org/wiki/Dilemme_biais-variance](https://fr.wikipedia.org/wiki/Dilemme_biais-variance) qui est référencé dans le cours, déduire une valeur de $\lambda$ qui réalise le bon compromis entre biais du modèle et variance du modèle. On pourra faire les mêmes calculs que précédemment avec chaque valeur $\lambda$  du vecteur de valeurs $\lambda  = [0,0.5,1,...,5]$ et obtenir une série de courbes comme décrites ci-dessous :

<img src="tp3-enonce-fig3.png" width="600"/>

Tracer les courbes de calcul du biais et de la variance du modèle pour chacune de ces valeurs et analyser le résultat pour conclure.

Rappel de la formule de décomposition de l’erreur décrite dans le cours :


- le biais se calcule par 

$$
B(h(x))= E[(h-f)(x)]
$$ 

- la variance se calcule par 

$$
Var(h(x))=E[h^2(x)]-(E[h(x)])^2
$$

On définit ensuite l'*erreur de prédiction attendue* par

$$
Err(h(x)) = B^2(h(x)) + Var(h(x)) + \sigma^2
$$

où $E$ correspondra ici à l’espérance mathématique sur l’ensemble des exemples d’apprentissage utilisé. 

L’intersection des courbes pour ces deux grandeurs en fonction de $\lambda$ ainsi que l’erreur globale indiquera la valeur du compromis biais-variance.

**Remarque** : Comme précisé dans le cours, l’analyse biais-variance se fait généralement sur plusieurs échantillonages ou jeux de données x (on en a qu’un seul ici car il y a trop peu de points expérimentaux) à partir desquels on obtient autant de fonctions hypothèses sur lesquels on doit moyenner les résultats. Le cas présenté est donc trop réduit pour cette analyse. Le résultat obtenu est de ce fait peu précis, voire peu fiable ici.

<img src="tp3-enonce-fig4.png" width="600"/>

## Éléments de calcul du compromis biais-variance proposé

Le Biais défini par la formule

$$
B(h(x))= E[(h-f)(x)]
$$

se calcule en faisant la moyenne (l'espérance se simplifie en moyenne, comme expliqué dans le cours) des coefficients du vecteur $y-h(x)$. Ce calcul s'obtient en appelant la fonction `mean`de `numpy`, à laquelle on passera en paramètre le vecteur/matrice $y-h(x)$.

La variance se calcule par la formule 
$$
Var[h(x)] = E[(h(x))^2] - (E[h(x)])^2
$$

Là encore on simplifie l'espérance mathématique $E$ par un simple calcul de moyenne. 

L'expression $Var[h(x)]$ se calculera simplement en appelant la fonction `var`de `numpy`, à laquelle on passera en paramètre le vecteur/matrice $h(x)$. 

Comme cela est mentionné, ce calcul de variance est une "interprétation" simplifiée de la définition du cours qui nécessite de mesurer la dispersion des valeurs de $h(x)$ par rapport à un ensemble d'échantillonages que l'on ne fait pas ici dans le TP. Calculer la "vraie" variance nécessiterait de mettre en place cet échantillonage et compliquerait singulièrement le TP. L'"interprétation" fait pour cet exercice est simple à calculer et produit des résultats exploitables et semblables au comportement de la "vraie" variance pour les données fournies.

Les courbes du biais et de la variance sont tracées dans cet exercice en prenant en abscisses les valeurs de $\lambda$. Mais comme la "complexité" du modèle est plus grande lorsque $\lambda$ est petit, les sens de variations des courbes seront à l'inverse de la figure donnée en cours. Ici la courbe du biais va croitre alors que celle de la variance va décroitre.

## Reprise du développement en utilisant la bibliothèque `scikit-learn`
Reprendre le programme en Python en remplaçant les fonctions de calcul de régression par les fonctions prédéfinies de la bibliothèse `scikit-learn` qui implémente les techniques de régularisation connues dont celle de Ridge sur laquelle s'appuie la méthode proposée dans le TP.